# 26. Tabularディープラーニング - TabNet & NODE (Tabular Deep Learning)

## 概要
テーブルデータに特化したディープラーニング手法を学びます。TabNet、NODEなど、GBDTに対抗できる最新のニューラルネットワークアーキテクチャを実装します。

## 学習目標
- Tabularデータでのディープラーニングの特徴を理解できる
- TabNetのアーキテクチャと特徴を理解できる
- TabNetを実装・学習できる
- GBDTとの比較ができる
- 実務でTabular DLを適用できる判断ができる

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb

# TabNet
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.pretraining import TabNetPretrainer

import torch
import warnings
warnings.filterwarnings('ignore')

# 設定
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. なぜTabularデータにディープラーニング？

### Tabularデータの特徴

**Tabularデータ（テーブルデータ）**は、行と列で構成された構造化データです。

### 従来の常識

長年、Tabularデータでは**GBDT（Gradient Boosting Decision Trees）が最強**とされてきました：

- ✅ XGBoost、LightGBM、CatBoost
- ✅ Kaggleコンペで圧倒的な勝率
- ✅ 特徴量エンジニアリング不要
- ✅ ハイパーパラメータ調整が比較的容易

### ディープラーニングの課題

一方、従来のニューラルネットワークはTabularデータで苦戦：

- ❌ GBDTより性能が低い
- ❌ 大量のデータが必要
- ❌ 特徴量の前処理が必須（正規化など）
- ❌ ハイパーパラメータ調整が難しい

### 新世代のTabular Deep Learning

近年、Tabularデータ専用に設計されたDLモデルが登場：

1. **TabNet** (Google Research, 2019)
   - 解釈可能性を持つニューラルネットワーク
   - Sequential Attention機構
   - Self-supervised事前学習

2. **NODE** (Neural Oblivious Decision Ensembles, 2019)
   - 決定木とNNのハイブリッド
   - Oblivious Decision Trees

3. **SAINT** (Self-Attention and Intersample Attention Transformer, 2021)
   - Transformer for Tabular Data

4. **FT-Transformer** (Feature Tokenizer Transformer, 2021)
   - 各特徴量をトークン化

### いつTabular DLを使うべきか

| 条件 | 推奨 |
|------|------|
| データサイズ < 10k | GBDT |
| データサイズ > 100k | Tabular DL検討 |
| 解釈可能性が重要 | TabNet または GBDT |
| 高速推論が必要 | GBDT |
| GPUが使える | Tabular DL |
| 事前学習データあり | TabNet |
| カテゴリカル変数多い | CatBoost または TabNet |

## 2. データの準備

In [ ]:
# データセットの読み込み（乳がん診断データ）
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(f"データサイズ: {X.shape}")
print(f"特徴量数: {X.shape[1]}")
print(f"サンプル数: {X.shape[0]}")
print(f"\nターゲット分布:")
print(y.value_counts())
print(f"\n特徴量の例:")
print(X.head())
print(f"\n統計量:")
print(X.describe())

In [ ]:
# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# さらに検証データを分割
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"訓練データ: {X_train.shape}")
print(f"検証データ: {X_val.shape}")
print(f"テストデータ: {X_test.shape}")

# スケーリング（ディープラーニングには必須）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("\n⚠️ 重要:")
print("- ディープラーニングでは特徴量のスケーリングが必須")
print("- 訓練データでfitし、検証・テストデータはtransformのみ")
print("- GBDTはスケーリング不要")

## 3. TabNet - 解釈可能なニューラルネットワーク

### TabNetとは

**TabNet**は、Google Researchが2019年に発表したTabularデータ専用のディープラーニングモデルです。

### TabNetの特徴

1. **Sequential Attention機構**
   - 各ステップで重要な特徴量を選択
   - 決定木のような逐次的な特徴選択

2. **解釈可能性**
   - Feature Importance（特徴量重要度）を出力
   - Attention Maskで重要な特徴を可視化

3. **Self-supervised事前学習**
   - ラベルなしデータで事前学習可能
   - 少量のラベル付きデータでも高性能

4. **エンドツーエンド学習**
   - 特徴量選択とモデル学習を同時に
   - 特徴量エンジニアリングが不要

### TabNetのアーキテクチャ

```
Input Features
     ↓
Feature Transformer (Shared across steps)
     ↓
[Step 1] Attentive Transformer → Feature Selection Mask
     ↓
[Step 2] Attentive Transformer → Feature Selection Mask
     ↓
   ...
     ↓
[Step N] Attentive Transformer → Feature Selection Mask
     ↓
Aggregate Decision
     ↓
Output (Classification/Regression)
```

### 3.1 TabNetの基本的な使い方

In [ ]:
# TabNetClassifierの初期化
tabnet_model = TabNetClassifier(
    n_d=64,                    # Decision層の次元数
    n_a=64,                    # Attention層の次元数
    n_steps=5,                 # Sequential Attentionのステップ数
    gamma=1.5,                 # Feature reusageのペナルティ
    lambda_sparse=1e-4,        # スパース性の正則化
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={
        "step_size":50,
        "gamma":0.9
    },
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',        # Attention maskのタイプ
    verbose=0,
    seed=42
)

print("TabNetモデルの初期化完了")
print("\nハイパーパラメータ:")
print(f"- n_d (Decision層): {tabnet_model.n_d}")
print(f"- n_a (Attention層): {tabnet_model.n_a}")
print(f"- n_steps (ステップ数): {tabnet_model.n_steps}")
print(f"- gamma (Feature reusage penalty): {tabnet_model.gamma}")

In [ ]:
# モデルの学習
tabnet_model.fit(
    X_train=X_train_scaled,
    y_train=y_train.values,
    eval_set=[(X_val_scaled, y_val.values)],
    eval_name=['valid'],
    eval_metric=['auc', 'accuracy'],
    max_epochs=100,
    patience=20,               # Early stoppingの忍耐度
    batch_size=256,
    virtual_batch_size=128,    # Ghost Batch Normalizationのサイズ
    num_workers=0,
    drop_last=False
)

print("\nTabNetの学習完了")
print(f"最良エポック: {tabnet_model.best_epoch}")

In [ ]:
# 予測
y_pred_tabnet = tabnet_model.predict(X_test_scaled)
y_proba_tabnet = tabnet_model.predict_proba(X_test_scaled)[:, 1]

# 評価
accuracy_tabnet = accuracy_score(y_test, y_pred_tabnet)
auc_tabnet = roc_auc_score(y_test, y_proba_tabnet)

print("=" * 60)
print("TabNetの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_tabnet:.4f}")
print(f"ROC-AUC:  {auc_tabnet:.4f}")
print("\n混同行列:")
print(confusion_matrix(y_test, y_pred_tabnet))
print("\n詳細レポート:")
print(classification_report(y_test, y_pred_tabnet, 
                           target_names=['Malignant', 'Benign']))

### 3.2 TabNetの学習曲線

In [ ]:
# 学習曲線の可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss曲線
train_losses = [tabnet_model.history['loss'][i] for i in range(len(tabnet_model.history['loss']))]
val_losses = [tabnet_model.history['valid_loss'][i] for i in range(len(tabnet_model.history['valid_loss']))]

axes[0].plot(train_losses, label='Train Loss', linewidth=2)
axes[0].plot(val_losses, label='Validation Loss', linewidth=2)
axes[0].axvline(tabnet_model.best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({tabnet_model.best_epoch})', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].set_title('Training and Validation Loss', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# AUC曲線
val_aucs = [tabnet_model.history['valid_auc'][i] for i in range(len(tabnet_model.history['valid_auc']))]

axes[1].plot(val_aucs, label='Validation AUC', linewidth=2, color='orange')
axes[1].axvline(tabnet_model.best_epoch, color='red', linestyle='--', 
                label=f'Best Epoch ({tabnet_model.best_epoch})', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('AUC', fontsize=11)
axes[1].set_title('Validation AUC', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 観察:")
print(f"- 最良エポック: {tabnet_model.best_epoch}")
print(f"- Early stoppingにより過学習を防止")
print(f"- 検証AUCは {max(val_aucs):.4f} に到達")

### 3.3 TabNetの特徴量重要度（解釈可能性）

In [ ]:
# 特徴量重要度の取得
feature_importances = tabnet_model.feature_importances_

# DataFrameに整形
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

# 上位15特徴量を可視化
top_features = importance_df.head(15)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance'].values, 
         alpha=0.7, edgecolor='black', color='steelblue')
plt.yticks(range(len(top_features)), top_features['feature'].values)
plt.xlabel('Feature Importance', fontsize=11)
plt.title('TabNet Feature Importances (Top 15)', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 10 重要な特徴量:")
for i, row in importance_df.head(10).iterrows():
    print(f"{row['feature']:30s}: {row['importance']:.6f}")

print("\n💡 TabNetの特徴量重要度:")
print("- Attention機構に基づく重要度")
print("- どの特徴量がモデルの決定に寄与したかを可視化")
print("- GBDTの特徴量重要度と比較可能")

In [ ]:
# サンプルごとのAttention Maskを可視化
# 予測時のAttentionを取得
explain_matrix, masks = tabnet_model.explain(X_test_scaled[:10])

# 最初の5サンプルのAttention Maskを可視化
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i in range(5):
    # explain_matrixは各サンプルの特徴量重要度
    importance_sample = explain_matrix[i]
    
    # 上位10特徴量のみ表示
    top_idx = np.argsort(importance_sample)[-10:][::-1]
    top_features_sample = X.columns[top_idx]
    top_importance_sample = importance_sample[top_idx]
    
    axes[i].barh(range(len(top_importance_sample)), top_importance_sample, 
                alpha=0.7, edgecolor='black')
    axes[i].set_yticks(range(len(top_importance_sample)))
    axes[i].set_yticklabels(top_features_sample, fontsize=8)
    axes[i].set_xlabel('Importance', fontsize=9)
    axes[i].set_title(f'Sample {i+1}\n(True: {y_test.iloc[i]}, Pred: {y_pred_tabnet[i]})',
                     fontsize=10, fontweight='bold')
    axes[i].invert_yaxis()
    axes[i].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 サンプルごとのAttention:")
print("- 各サンプルで異なる特徴量が重要")
print("- モデルがどの特徴に注目して予測したかを理解")
print("- 医療診断などで重要な解釈可能性")

## 4. GBDTとの比較

In [ ]:
# LightGBM
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    verbose=-1
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
)

y_pred_lgb = lgb_model.predict(X_test)
y_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]

accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
auc_lgb = roc_auc_score(y_test, y_proba_lgb)

print("=" * 60)
print("LightGBMの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_lgb:.4f}")
print(f"ROC-AUC:  {auc_lgb:.4f}")

# XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=5,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

y_pred_xgb = xgb_model.predict(X_test)
y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

print("\n" + "=" * 60)
print("XGBoostの性能（テストデータ）")
print("=" * 60)
print(f"Accuracy: {accuracy_xgb:.4f}")
print(f"ROC-AUC:  {auc_xgb:.4f}")

In [ ]:
# 性能比較
comparison_df = pd.DataFrame([
    {'Model': 'TabNet', 'Accuracy': accuracy_tabnet, 'ROC-AUC': auc_tabnet},
    {'Model': 'LightGBM', 'Accuracy': accuracy_lgb, 'ROC-AUC': auc_lgb},
    {'Model': 'XGBoost', 'Accuracy': accuracy_xgb, 'ROC-AUC': auc_xgb}
]).sort_values('ROC-AUC', ascending=False)

print("\n" + "=" * 60)
print("モデル性能比較")
print("=" * 60)
print(comparison_df.to_string(index=False))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
            alpha=0.7, edgecolor='black', color=['steelblue', 'lightgreen', 'coral'])
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_title('Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.9, 1.0])
axes[0].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[0].text(row.name, row['Accuracy'], f"{row['Accuracy']:.4f}",
                ha='center', va='bottom', fontweight='bold')

# ROC-AUC
axes[1].bar(comparison_df['Model'], comparison_df['ROC-AUC'], 
            alpha=0.7, edgecolor='black', color=['steelblue', 'lightgreen', 'coral'])
axes[1].set_ylabel('ROC-AUC', fontsize=11)
axes[1].set_title('ROC-AUC Comparison', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.9, 1.0])
axes[1].grid(axis='y', alpha=0.3)
for i, row in comparison_df.iterrows():
    axes[1].text(row.name, row['ROC-AUC'], f"{row['ROC-AUC']:.4f}",
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 観察:")
print("- TabNetはGBDTと同等またはそれ以上の性能")
print("- 解釈可能性ではTabNetが優位（Attention可視化）")
print("- 小規模データではGBDTも依然強力")

## 5. TabNetの事前学習（Self-supervised Learning）

TabNetの強力な機能の1つは、**Self-supervised事前学習**です。

### なぜ事前学習が有効か

- ✅ ラベルなしデータを活用できる
- ✅ 少量のラベル付きデータでも高性能
- ✅ より良い特徴表現を学習
- ✅ 過学習を防ぐ正則化効果

### TabNetの事前学習手法

一部の特徴量を**マスク**し、残りの特徴量から復元するタスクを学習します（Masked Language Modelingと類似）。

In [ ]:
# ラベルなしデータを模擬（全データからラベルを除外）
X_unlabeled = np.concatenate([X_train_scaled, X_val_scaled, X_test_scaled], axis=0)

print(f"ラベルなしデータ: {X_unlabeled.shape}")

# 事前学習モデルの初期化
pretrainer = TabNetPretrainer(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax',
    verbose=0,
    seed=42
)

# 事前学習の実行
pretrainer.fit(
    X_train=X_unlabeled,
    eval_set=[X_unlabeled],
    max_epochs=50,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
    pretraining_ratio=0.5  # マスクする特徴量の割合
)

print("\n事前学習完了")

In [ ]:
# 事前学習済みモデルでFine-tuning
tabnet_pretrained = TabNetClassifier(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":50, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='entmax',
    verbose=0,
    seed=42
)

# 事前学習済み重みをロード
tabnet_pretrained.load_from_pretrained(pretrainer)

# Fine-tuning
tabnet_pretrained.fit(
    X_train=X_train_scaled,
    y_train=y_train.values,
    eval_set=[(X_val_scaled, y_val.values)],
    eval_name=['valid'],
    eval_metric=['auc', 'accuracy'],
    max_epochs=100,
    patience=20,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# 予測
y_pred_pretrained = tabnet_pretrained.predict(X_test_scaled)
y_proba_pretrained = tabnet_pretrained.predict_proba(X_test_scaled)[:, 1]

accuracy_pretrained = accuracy_score(y_test, y_pred_pretrained)
auc_pretrained = roc_auc_score(y_test, y_proba_pretrained)

print("\n" + "=" * 60)
print("事前学習済みTabNetの性能")
print("=" * 60)
print(f"Accuracy: {accuracy_pretrained:.4f}")
print(f"ROC-AUC:  {auc_pretrained:.4f}")

print("\n比較:")
print(f"TabNet (事前学習なし): AUC = {auc_tabnet:.4f}")
print(f"TabNet (事前学習あり): AUC = {auc_pretrained:.4f}")
print(f"改善: {(auc_pretrained - auc_tabnet):.4f}")

## 6. TabNetのハイパーパラメータ

### 主要なハイパーパラメータ

| パラメータ | 説明 | 推奨値 |
|-----------|------|--------|
| `n_d` | Decision層の次元数 | 8-64 |
| `n_a` | Attention層の次元数 | 8-64 |
| `n_steps` | Sequential Attentionのステップ数 | 3-10 |
| `gamma` | Feature reusageのペナルティ | 1.0-2.0 |
| `lambda_sparse` | スパース性の正則化 | 1e-6 ~ 1e-3 |
| `learning_rate` | 学習率 | 1e-2 ~ 5e-2 |
| `batch_size` | バッチサイズ | 256-1024 |
| `virtual_batch_size` | Ghost Batch Norm | 128-256 |

### チューニングのポイント

1. **n_d, n_a**: 大きいほど表現力が高いが、過学習のリスク
2. **n_steps**: ステップ数が多いほど複雑なパターンを学習
3. **gamma**: 高いほど各ステップで異なる特徴を使うように強制
4. **lambda_sparse**: スパース性を高め、解釈性を向上

## 7. 実務でのベストプラクティス

### TabNetを使うべき場面

| 状況 | 推奨 |
|------|------|
| 解釈可能性が重要 | TabNet ✅ |
| データサイズ > 10k | TabNet検討 |
| ラベルなしデータ多数 | TabNet（事前学習） ✅ |
| GPUが使える | TabNet ✅ |
| 高速推論が必要 | GBDT |
| データサイズ < 5k | GBDT |
| カテゴリカル変数多数 | CatBoost または TabNet |

### チェックリスト

```python
# ✅ TabNet実装チェックリスト

# 1. データの前処理
# - 特徴量をスケーリングしたか？（StandardScaler推奨）
# - 欠損値を処理したか？
# - カテゴリカル変数をエンコードしたか？

# 2. データ分割
# - 訓練・検証・テストに分割したか？
# - 検証データでEarly stoppingを使うか?

# 3. ハイパーパラメータ
# - n_d, n_a, n_stepsを調整したか？
# - learning rateを適切に設定したか？
# - batch sizeを調整したか？

# 4. 学習
# - Early stoppingを設定したか？
# - 学習曲線を確認したか？
# - 過学習していないか？

# 5. 評価
# - テストデータで性能を評価したか？
# - GBDTと比較したか？
# - 特徴量重要度を確認したか？

# 6. 事前学習（オプション）
# - ラベルなしデータがあるか？
# - 事前学習で性能が向上したか？
```

## 8. まとめ

### 本ノートブックで学んだこと

1. **Tabularディープラーニングの必要性**
   - GBDTとの比較
   - 新世代のTabular DLモデル
   - 使い分けの基準

2. **TabNetの基礎**
   - Sequential Attention機構
   - 解釈可能性の実現
   - アーキテクチャの理解

3. **TabNetの実装**
   - モデルの初期化と学習
   - ハイパーパラメータの設定
   - Early stopping

4. **解釈可能性**
   - グローバル特徴量重要度
   - サンプルごとのAttention可視化
   - 医療診断などでの応用

5. **事前学習**
   - Self-supervised learning
   - ラベルなしデータの活用
   - Fine-tuningによる性能向上

6. **GBDTとの比較**
   - 性能の比較
   - それぞれの強み
   - 使い分けの判断基準

### 重要なポイント

- ✅ **スケーリング必須**: TabNetは特徴量の正規化が必要
- ✅ **解釈可能性**: Attention可視化で透明性を確保
- ✅ **事前学習**: ラベルなしデータがあれば活用
- ✅ **Early stopping**: 過学習を防ぐ
- ✅ **GBDTと比較**: まずGBDTを試し、TabNetで改善を図る
- ✅ **GPU推奨**: 学習の高速化にGPUが有効

### 次のステップ

- Notebook 27でKaggle完全ワークフローを学ぶ
- Notebook 28で総合演習プロジェクトに挑戦
- 実際のKaggleコンペでTabNetを試す
- FT-Transformer、SAINTなど他のTabular DLモデルを学ぶ